In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta

In [2]:
# Carga información y se obtiene la info de columnas y número de registros totales
shark = pd.read_csv('../csv/attacks.csv', encoding='latin-1')
shark.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [3]:
shark.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [4]:
shark.shape

(25723, 24)

In [5]:
# Eliminar todos los registros que tengan valores nulos en todas sus columnas
shark.dropna(how='all', inplace = True)
shark.shape

(8703, 24)

In [6]:
# Se eliminan las columnas que no tienen información
shark.drop(['href formula','Case Number.1', 'Case Number.2','Unnamed: 22','Unnamed: 23'],axis=1, inplace = True)

In [7]:
# Nombres originales
old_columns_names = shark.columns

In [8]:
# Nombres_nuevos
new_columns_names = [re.sub(r'\s|\.','_',name.strip()) for name in old_columns_names]
new_columns_names = [re.sub(r'\(|\)|\/','',name.strip()) for name in new_columns_names]
new_columns_names

['Case_Number',
 'Date',
 'Year',
 'Type',
 'Country',
 'Area',
 'Location',
 'Activity',
 'Name',
 'Sex',
 'Age',
 'Injury',
 'Fatal_YN',
 'Time',
 'Species',
 'Investigator_or_Source',
 'pdf',
 'href',
 'original_order']

In [9]:
rename_names = dict(zip(old_columns_names, new_columns_names))
rename_names

{'Case Number': 'Case_Number',
 'Date': 'Date',
 'Year': 'Year',
 'Type': 'Type',
 'Country': 'Country',
 'Area': 'Area',
 'Location': 'Location',
 'Activity': 'Activity',
 'Name': 'Name',
 'Sex ': 'Sex',
 'Age': 'Age',
 'Injury': 'Injury',
 'Fatal (Y/N)': 'Fatal_YN',
 'Time': 'Time',
 'Species ': 'Species',
 'Investigator or Source': 'Investigator_or_Source',
 'pdf': 'pdf',
 'href': 'href',
 'original order': 'original_order'}

In [10]:
shark.rename(columns=rename_names, inplace = True)
shark.columns

Index(['Case_Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal_YN', 'Time',
       'Species', 'Investigator_or_Source', 'pdf', 'href', 'original_order'],
      dtype='object')

In [11]:
index_to_clean = shark[(shark['Case_Number'] == '0') | (shark['Case_Number'] == 'xx')].index
len(index_to_clean)
shark.drop(index_to_clean, inplace=True)

In [12]:
# Para evitar los NAN
shark.Type = shark.Type.fillna('Unspecified')
shark.Country = shark.Country.fillna('').str.upper()

In [13]:
# Reemplaza caracteres especiales del país
shark.Country = shark.Country.str.replace('/','').replace('?','')
shark.Country = shark.Country.str.replace('?','')
shark.Injury  = shark.Injury.str.lower() 

In [14]:
# Limpiando la columna Sex
shark.Sex = shark.Sex.str.strip()
shark.Sex = shark.Sex.str.replace('.','').replace('lli','').fillna('')

In [15]:
# Convirtiendo a numero entero
shark.Year= shark.Year.fillna(0).astype('int')
shark.original_order = shark.original_order.astype('int')

In [16]:
def get_date(fecha):
    
    if pd.isnull(fecha):
        fecha = '0001'

    if len(fecha.strip()) >= 10:
        # buscar año, mes, día
        anio = fecha[:4]
        if fecha[5:7] != '00':
            mes  = fecha[5:7]
        else:
            mes  = '01'
        if fecha[8:10] != '00':
            dia  = fecha[8:10]
        else:
            dia  = '01'
            
        fecha_clean = dia + '/'+ mes +'/'+ anio
        
    else:
        anio = re.findall(r'\d{4}',fecha)[0]
        mes  = '01'
        dia  = '01'
        fecha_clean = dia + '/'+ mes +'/'+ anio
    
    return fecha_clean

In [17]:
# Arregla la fecha 
shark['Date'] = shark['Case_Number'].map(get_date)

In [18]:
def get_age(age):
    if pd.isnull(age):
        ages = '0'
    elif age.find('month') > 0:
        ages = re.findall(r'\d+',age)
        age_b = []
        for a in ages:
            age_b.append(str(int(a)/12))
            
        ages = age_b
    else:
        ages = re.findall(r'\d+',age)
    
    if len(ages) > 0:
        return ",".join(ages)
    else:
        return '0'
    

In [19]:
# Arreglar la columna edad
shark['Age'] = shark['Age'].map(get_age)

In [20]:
null_columns = shark.columns[shark.isnull().any()]
null_columns

for col in null_columns:
    shark[col] = shark[col].fillna('')


In [21]:
def get_time(time):
    if pd.isnull(time):
        time_clean = '00:00'
    else:
        time_list = re.findall(r'\d+',time)
        if len(time_list) > 1:
            time_clean = time_list[0] + ':' + time_list[1]
        elif len(time_list) == 0:
            time_clean = '00:00'
        elif len(time_list) == 1:
            if len(time_list[0]) == 4:
                time_clean = time_list[0][0:2] + ':' + time_list[0][2:4]
            else:
                time_clean = time_list[0] + ':00'
            
            
    return time_clean
    

In [22]:
shark['Time'] = shark['Time'].map(get_time)

In [23]:
shark.to_csv('../csv/attacks_clean.csv', index = False)